In [470]:
import numpy as np
import sympy as sym
from numpy.random import randn
np.set_printoptions(threshold=np.inf)


### Values

In [471]:
m = 5
sigma = 2
sigma2 = 2*sigma**2
w = randn(m) / sigma
b = np.random.rand(m)*2*np.pi

n = 5
x = randn(n)/2
z = randn(n)/2 # function y

### Gaussian Kernel

In [472]:
def k_gauss(x,z):
    """R^n x R^n -> R - should be one number
    # vector is multi-dimensional

    Args:
        x (float): real value numbers
        z (float): real value numbers

    Returns:
        (vector with length n): a scalar shift-invariant Gaussian kernel
    """
    return np.exp(-((np.linalg.norm(x-z))**2)/(sigma2)) 

print(k_gauss(x,z))

0.7273991349482477


###  Scalar kernel

In [473]:
def k_approx(k, x, z):
    """scalar kernel

    Args:
        k (kernel): to test with different kernels

    Returns:
        approximated kernel: sum of feature vectors
    """
    n = len(x)
    Kxz = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            Kxz[i, j] = k(x[i], z[j])
    return Kxz

### Approximation 1

In [474]:
def psi_approx1(x):
   """
   Args:
       x (vector): vector with length n

   Returns:
       vector: vector with length n
   """
   return np.sqrt(2)*np.cos(w*x + b)

print(psi_approx1(x))

[-0.84878461 -1.21700176 -0.73624732 -1.11227512 -0.25579489]


### Approximation 2

In [475]:
def psi_cos(x):
    return 1/np.sqrt(m) * np.cos(w.T@x)

def psi_sin(x):
    return 1/np.sqrt(m) * np.sin(w.T@x)

def add_psi_cos_sin(f, g, x):
    """
        [1 2 3] and [1 2 3] -> [1 1 2 2 3 3]
    """
    new_list = []
    for i in range (m):
        new_list.append(f(x))
        new_list.append(g(x))
    return new_list

print("cos:", psi_cos(x))
print("sin:", psi_sin(x))
print("\n added together:", add_psi_cos_sin(psi_cos, psi_sin, x))
print("w:", w, w.T)
print("x:", x)

def K_approx2(x, z):
    """The scalar kernel

    Args:
        x (float): random generated numbers
        z (float): random generated numbers

    Returns:
        (float): a number, which is the scalar kernel
    """
    return np.sum(psi_cos(x) * psi_cos(z)) + np.sum(psi_sin(x) * psi_sin(z))

print(K_approx2(x,z))

cos: 0.40851470823866914
sin: -0.18197728746377942

 added together: [0.40851470823866914, -0.18197728746377942, 0.40851470823866914, -0.18197728746377942, 0.40851470823866914, -0.18197728746377942, 0.40851470823866914, -0.18197728746377942, 0.40851470823866914, -0.18197728746377942]
w: [0.60771043 0.1631019  0.4867397  0.149675   0.02618134] [0.60771043 0.1631019  0.4867397  0.149675   0.02618134]
x: [-0.15882639  0.14587088 -0.60062325 -0.49526923  0.76896482]
0.19954429511041116


### Matrix kernels

In [476]:
# Matrix kernel
def K(k, x, z):
    return k(x,z) * np.eye(m)

# Testing for gaussian kernel
gauss_sep_ker = K(k_gauss, x, z)
print(gauss_sep_ker)

# Testing for approximation with RFF (random fourier features)
print("\n", "matrix kernel")
matrix_ker = K(K_approx2, x, z)
print(matrix_ker)

[[0.72739913 0.         0.         0.         0.        ]
 [0.         0.72739913 0.         0.         0.        ]
 [0.         0.         0.72739913 0.         0.        ]
 [0.         0.         0.         0.72739913 0.        ]
 [0.         0.         0.         0.         0.72739913]]

 matrix kernel
[[0.1995443 0.        0.        0.        0.       ]
 [0.        0.1995443 0.        0.        0.       ]
 [0.        0.        0.1995443 0.        0.       ]
 [0.        0.        0.        0.1995443 0.       ]
 [0.        0.        0.        0.        0.1995443]]


### Approximate function f